# AIVLE스쿨 1차 미니프로젝트: 서울시 생활정보 기반 대중교통 수요 분석

<img src = "https://github.com/Jangrae/img/blob/master/bus.png?raw=true" width=800, align="left"/>

- 본 과정에서는 서울시 생활정보 데이터를 기반으로 데이터 분석을 시도하고 주어진 문제를 해결할 것입니다.
- 우리는 공공 데이터를 활용해 버스노선 관련 인사이트를 도출하여 유용한 정책을 세우는데 도움을 주고자 합니다.

### 로컬 수행(Anaconda)
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [1]:
# 기본 경로
path = ''

### 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [2]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

### 한글 폰트 표시를 위해 설치합니다.

In [3]:
# 한글 표시를 위한 라이브러리 설치
!pip install koreanize_matplotlib -q

### 필요한 라이브러리를 불러옵니다.

In [4]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import scipy.stats as spst
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

## ▣ 우리가 풀어야 하는 문제는 무엇인가요?

- 제공 데이터
    - 버스 승하차 및 운행 노선 데이터
    - 구별 유동인구 데이터
    - 구별 주민 등록 데이터
    - 구별 업종 등록 정보
    
#### 위 제공 데이터를 분석하여 버스 노선 추가 필요 대상 지역(구 단위)을 선정합니다.

# 1. 데이터 불러오기

- 이전 과정에서 저장한 다음 4개의 파일을 불러와 데이터프레임으로 선언합니다.
    - 구별 버스 정류장 데이터(df_seoul_bus_station.csv) → df_seoul_bus_station
    - 구별 유동인구 데이터(df_seoul_moving.csv) → df_seoul_moving
    - 구별 상권 데이터(df_seoul_business.csv) → df_seoul_business
    - 구별 인구 데이터(df_seoul_people.csv) → df_seoul_people

In [5]:
# 구별 버스 정류장 데이터불러오기
df_seoul_bus_station = pd.read_csv(path + 'df_seoul_bus_station.csv')

# 확인
df_seoul_bus_station.head()

,자치구,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수
0,강남구,502,99,9113021,8640353,135.163908,128.153318
1,강동구,376,24,3306588,3242901,94.638885,92.816079
2,강북구,410,71,5244436,5077041,138.244306,133.831743
3,강서구,565,54,5244615,5218001,93.106837,92.634362
4,관악구,465,82,7691243,7850254,147.777793,150.832994


In [6]:
# 구별 유동인구 데이터 불러오기
df_seoul_moving = pd.read_csv(path + 'df_seoul_moving.csv')

# 확인
df_seoul_moving.head()

,자치구,평균 이동 시간(분),이동인구(합),총 이동 시간,총 이동인구
0,강남구,65.220874,89.880083,37563832,51766253.76
1,강동구,71.135695,67.088111,27212889,25664489.79
2,강북구,65.258796,38.290780,24201877,14200518.80
3,강서구,70.107829,59.547527,30772920,26137612.75
4,관악구,63.064536,47.985346,27629141,21022812.01


In [7]:
#  구별 상권 데이터 불러오기
df_seoul_business = pd.read_csv(path + 'df_seoul_business.csv')

# 확인
df_seoul_business.head()

,자치구,한식 일반 음식점업,커피전문점,기타주점업
0,종로구,5912,3613,1413
1,중구,6414,3542,982
2,용산구,2987,2076,1332
3,성동구,2785,1859,621
4,광진구,2861,1887,1577


In [8]:
# 구별 인구 데이터 불러오기
df_seoul_people = pd.read_csv(path + 'df_seoul_people.csv')

# 확인
df_seoul_people.head()

,자치구,남자,여자,합계
0,종로구,71654,78390,150044
1,중구,63365,68392,131757
2,용산구,105480,113343,218823
3,성동구,136779,146555,283334
4,광진구,166541,183141,349682


# 2. 데이터 합치기

- 위에서 선언한 네 개의 데이터프레임을 '자치구'열을 기준으로 조인(merge)합니다.
- 조인 결과를 df 데이터프레임으로 선언합니다.

In [9]:
# 네 개의 데이터프레임 조인(megre)
df = pd.merge(df_seoul_bus_station, df_seoul_moving, how='inner', on='자치구')
df = pd.merge(df, df_seoul_business, how='inner', on='자치구')
df = pd.merge(df, df_seoul_people, how='inner', on='자치구')

In [10]:
# 확인
df.head()

,자치구,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수,평균 이동 시간(분),이동인구(합),총 이동 시간,총 이동인구,한식 일반 음식점업,커피전문점,기타주점업,남자,여자,합계
0,강남구,502,99,9113021,8640353,135.163908,128.153318,65.220874,89.880083,37563832,51766253.76,11486,7273,2528,268993,293762,562755
1,강동구,376,24,3306588,3242901,94.638885,92.816079,71.135695,67.088111,27212889,25664489.79,3388,1660,1349,228197,239705,467902
2,강북구,410,71,5244436,5077041,138.244306,133.831743,65.258796,38.290780,24201877,14200518.80,2663,1095,1275,139260,150567,289827
3,강서구,565,54,5244615,5218001,93.106837,92.634362,70.107829,59.547527,30772920,26137612.75,4584,2530,1709,270025,294460,564485
4,관악구,465,82,7691243,7850254,147.777793,150.832994,63.064536,47.985346,27629141,21022812.01,3689,2049,1672,248571,248566,497137


# 3. 가설 수립
$ \huge X \rightarrow Y $
- 가설을 수립해 보세요. (3개 이상)
- 미니프로젝트 진행의 편의상 가설은 이변량 분석을 염두에 두고 설정해 주세요.
- 다양한 가설이 가능하나, 프로젝트 주제에 맞게 '노선 수'에 대한 가설도 포함해 주세요.

### 1) 가설 1: 승차총승객수 → 정류장수

- H0: 승차총승객수와 정류장수는 연관이 없다
- H1: 승차총승객수와 정류장수는 연관이 있다

### 2) 가설 2: 총 이동 시간 → 노선수

- H0: 총 이동 시간과 노선수는 연관이 없다
- H1: 총 이동 시간과 노선수는 연관이 있다

### 3) 가설 3: 총 이동인구 → 정류장수

### 4) 가설 4: 평균 이동 시간(분) → 노선수

### 5) 가설 5: 커피전문점 → 노선수

# 4. 단변량 분석

- 변수의 비즈니스 의미  
- 숫자형, 범주형 구분  
- 전처리 사항
   - NaN 존재 여부 및 조치 방안  
   - 가변수화 대상여부  
- 기초통계량
- 분포  
- 위 정보로부터 파악한 내용(비즈니스) 정리  
- 추가 분석해볼 사항 도출 등

### 1)

### 2)

### 3)

# 5.이변량 분석

- 자료 종류에 맞게 $X \rightarrow Y$ 에 대해서 그래프(시각화)와 가설검정(수치화)를 수행하고 결과를 평가하세요.
- 가설 검정 시 다음 항목을 참조하여 수행하세요.
    - 유의수준: 5%
    - 숫자 → 숫자: 상관분석
    - 범주 → 범주: 카이제곱검정
    - 범주 → 숫자: t검정, 분산분석
    - 숫자 → 범주: 로지스틱 회귀모형을 통해, 회귀계수의 p-value로 검정을 수행합니다.

In [11]:
# 상관 계수
correlation_matrix = df.corr(numeric_only=True)
correlation_matrix.style.background_gradient()

,정류장수,노선수,승차총승객수,하차총승객수,승차평균승객수,하차평균승객수,평균 이동 시간(분),이동인구(합),총 이동 시간,총 이동인구,한식 일반 음식점업,커피전문점,기타주점업,남자,여자,합계
정류장수,1.000000,0.299961,0.575473,0.588017,-0.225378,-0.199972,0.350562,0.438203,0.466639,0.423217,0.179676,0.240494,0.344005,0.595026,0.627087,0.612568
노선수,0.299961,1.000000,0.730705,0.718799,0.512891,0.495674,-0.516482,0.050999,0.406300,0.271757,0.466473,0.480902,0.293330,-0.227867,-0.192799,-0.209958
승차총승객수,0.575473,0.730705,1.000000,0.996860,0.453681,0.459838,-0.018810,0.506222,0.663187,0.631463,0.645277,0.609624,0.570369,0.370442,0.385024,0.378566
하차총승객수,0.588017,0.718799,0.996860,1.000000,0.445184,0.461622,-0.013891,0.497578,0.658294,0.617499,0.616830,0.578460,0.561139,0.391217,0.403396,0.398121
승차평균승객수,-0.225378,0.512891,0.453681,0.445184,1.000000,0.991114,-0.412800,-0.138138,0.086712,0.033042,0.328258,0.263669,0.215395,-0.367552,-0.395727,-0.382742
하차평균승객수,-0.199972,0.495674,0.459838,0.461622,0.991114,1.000000,-0.407048,-0.138376,0.095668,0.027151,0.295287,0.227714,0.218286,-0.323916,-0.355350,-0.340738
평균 이동 시간(분),0.350562,-0.516482,-0.018810,-0.013891,-0.412800,-0.407048,1.000000,0.520569,0.084929,0.267037,0.017731,-0.042916,0.083641,0.674250,0.665601,0.670733
이동인구(합),0.438203,0.050999,0.506222,0.497578,-0.138138,-0.138376,0.520569,1.000000,0.830091,0.950544,0.737657,0.714513,0.599446,0.718017,0.738907,0.729928
총 이동 시간,0.466639,0.406300,0.663187,0.658294,0.086712,0.095668,0.084929,0.830091,1.000000,0.937032,0.831822,0.840665,0.694401,0.458313,0.495671,0.478411
총 이동인구,0.423217,0.271757,0.631463,0.617499,0.033042,0.027151,0.267037,0.950544,0.937032,1.000000,0.870272,0.861088,0.676822,0.572145,0.600313,0.587629


### 상관계수 (우리가 배운 기준)

- 0.5 ~ 1: 강한
- 0.2 ~ 0.5: 중간
- 0.1 ~ 0.2: 약간
- 0 ~ 0.1: 없음
- 음수 똑같음

### p-value 기준: 0.05 (5%)

- p-value가 0.05보다 크다 => 귀무가설을 기각 하지 못한다 => 대립가설(우리가 원하는 바)이 틀렸다
- 귀무가설을 기각하려면 우리가 싫어하는 귀무가설을 입증할 증거가 나올 확률이 5% 이하여야 함
- p-value가 그 귀무가설을 입증할 증거가 나올 확률을 의미함

### 1) 승차총승객수 → 정류장수

In [ ]:
# 승차총승객수와 정류장수 상관관계
result = spst.pearsonr(df['승차총승객수'], df['정류장수'])
print('* 상관계수:', result[0])
print('* p-value:', result[1])
# p-value가 0.05보다 작다 = 귀무가설 기각, 승차총승객수는 정류장 수와 상관관계가 있다.

### 2) 총 이동 시간 → 노선수

In [ ]:
# 총 이동 시간과 노선수 상관관계
result = spst.pearsonr(df['총 이동 시간'], df['노선수'])
print('* 상관계수:', result[0])
print('* p-value:', result[1])
# p-value가 0.05보다 작다 == 귀무가설을 기각 == 총 이동 시간은 노선수와 상관이 있다.

### 3) 커피전문점 → 노선수

In [ ]:
# 커피전문점과 노선수 상관관계
result = spst.pearsonr(df['커피전문점'], df['노선수'])
print('* 상관계수:', result[0])
print('* p-value:', result[1])
# p-value가 0.05보다 작다 == 귀무가설을 기각 == 커피전문점은 노선수와 상관이 있다.

### 4) HeatMap으로 시각화

In [ ]:
# 상관계수를 HeatMap으로 시각화
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(numeric_only=True),
            annot=True,
            cmap='Blues',
            cbar=False,
            fmt='.2f',
            square=True,
            annot_kws={'size': 9})
plt.show()

In [ ]:
# 위 차트를 통해 알게된 사실을 정리해봅시다.
# 1. 정류장 수가 많은 지역에서 승하차 승객 수가 많음. 이는 더 많은 정류장이 편리한 접근성을 제공하기 때문
# 2. 노선 수가 많은 지역은 대체로 승하차 승객 수도 많음. 이는 다양한 목적지에 대한 접근성이 높음을 의미함
# 3.

# 6. 관계 정리

### 1) 강한 관계의 열

In [ ]:
# 정류장수 - 하차총승객수
# 정류장수 - 승차총승객수
# 노선수 - 하차총승객수
# 노선수 - 승차총승객수 등

### 2) 중간 관계의 열

In [ ]:
# 노선수 - 커피전문점
# 노선수 - 한식 일반 음식점업 등

# 7. 솔루션 도출

- 위에서 확인된 내용을 기초로하여 노선(또는 정류장) 추가를 제안할 자치구를 확인하세요.

### 1) 비율1 = 노선수 / 승차총승객수

In [ ]:
# 노선수 / 승차총승객수
df['비율1'] = df['노선수'] / df['승차총승객수'] * 10000
df.sort_values(by=['비율1']).head()
# 강동구다 승차총승객수 대비 노선수가 적음

### 2) 비율2 = 노선수 / 커피전문점

In [ ]:
# 노선수 / 커피전문점
df['비율2'] = df['노선수'] / df['커피전문점'] * 100
df.sort_values(by=['비율2']).head()
# 강남구가 커피전문점 종사자수 대비 노선수가 적음

### 결론1: 어느 구에 버스 노선 또는 정류장이 제일 필요한가요?

In [ ]:
# 강남구, 강동구

### 결론2: 그 이유는 무엇인가요?

In [ ]:
# 이동 수요가 높은 지역이지만, 노선 수가 상대적으로 적음